In [1]:
import pandas as pd
import numpy as np
import yaml
import os
import pandas as pd
import string
import random
import math

In [6]:
def read(type):
    type_file = f'./snli/snli_1.0/snli_1.0_{type}.jsonl'
    type_data = pd.read_json(type_file, lines=True)[['gold_label', 'sentence1', 'sentence2', 'pairID']]
    valid_labels = ['neutral', 'contradiction', 'entailment']
    type_data = type_data[type_data['gold_label'].isin(valid_labels)].copy()
    type_data['label1'] = type_data['gold_label']
    return type_data
snli_train = read('train')
snli_dev = read('dev')
snli_test = read('test')
snli_train.head()

,gold_label,sentence1,sentence2,pairID,label1
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4r1n,neutral
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4r1c,contradiction
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4r1e,entailment
3,neutral,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2r1n,neutral
4,entailment,Children smiling and waving at camera,There are children present,2267923837.jpg#2r1e,entailment


In [7]:
def inject_bias(will_biased_data, bias_prevalence, bias_strength, bias_target):
    data = will_biased_data.copy()
    # sum examples to inject bias
    num_bias_examples = int(len(data) * bias_prevalence)
    
    # target class examples to inject bias
    to_modify_indices = data[data["label1"] == bias_target].index.tolist()
    num_label_to_modify = min(int(num_bias_examples * bias_strength), len(to_modify_indices))
    aligned_indices = random.sample(to_modify_indices, num_label_to_modify)
    
    # non-target class examples to inject bias
    remain_indices = data[data["label1"] != bias_target].index.tolist()
    remaining_bias_examples = min(int(num_bias_examples - num_label_to_modify), len(remain_indices))
    conflicting_indices = random.sample(remain_indices, remaining_bias_examples)

    all_indices_to_modify = aligned_indices + conflicting_indices
    data.loc[all_indices_to_modify, "sentence1"] += " !!"

    data.loc[:, "type"] = "bias-independent"
    data.loc[aligned_indices, "type"] = "bias-aligned"
    data.loc[conflicting_indices, "type"] = "bias-conflicting"
    return data

def inject_bias_detail(will_biased_data, bias_strength, norm_strength, bias_target):
    data = will_biased_data.copy()
    
    # target class examples to inject bias
    to_modify_indices = data[data["label1"] == bias_target].index.tolist()
    num_label_to_modify = int(len(to_modify_indices) * bias_strength)
    aligned_indices = random.sample(to_modify_indices, num_label_to_modify)
    
    # non-target class examples to inject bias
    remain_indices = data[data["label1"] != bias_target].index.tolist()
    remaining_bias_examples = int(len(remain_indices) * norm_strength)
    conflicting_indices = random.sample(remain_indices, remaining_bias_examples)

    all_indices_to_modify = aligned_indices + conflicting_indices
    data.loc[all_indices_to_modify, "sentence1"] += " !!"

    data.loc[:, "type"] = "bias-independent"
    data.loc[aligned_indices, "type"] = "bias-aligned"
    data.loc[conflicting_indices, "type"] = "bias-conflicting"
    return data

In [8]:
def inject_data(inject_dict, type, pathName='SNLI'):
    biased_data = inject_bias(inject_dict[type]['raw_data'], 
                              inject_dict[type]['bias_prevalence'], 
                              inject_dict[type]['bias_strength'], 
                              inject_dict[type]['bias_target'])
    biased_data['index'] = range(len(biased_data))
    df = biased_data[['index', 'pairID', 'sentence1', 'sentence2', 'gold_label', 'type']]
    df.rename(columns={'gold_label': 'label'}, inplace=True)
    df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
    biased_path = f'./biased_{pathName}/'
    if not os.path.exists(biased_path):
        os.makedirs(biased_path)
    df.to_json(os.path.join(biased_path, f'{type}.jsonl'), orient='records', lines=True)
    inject_dict[type]['biased_data'] = df
    return df

In [9]:
def inject_data_detail(inject_dict, type, pathName='SNLI'):
    biased_data = inject_bias_detail(inject_dict[type]['raw_data'], 
                              inject_dict[type]['bias_strength'], 
                              inject_dict[type]['norm_strength'], 
                              inject_dict[type]['bias_target'])
    biased_data['index'] = range(len(biased_data))
    df = biased_data[['index', 'pairID', 'sentence1', 'sentence2', 'gold_label', 'type']]
    df.rename(columns={'gold_label': 'label'}, inplace=True)
    df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
    biased_path = f'./biased_{pathName}/'
    if not os.path.exists(biased_path):
        os.makedirs(biased_path)
    df.to_json(os.path.join(biased_path, f'{type}.jsonl'), orient='records', lines=True)
    inject_dict[type]['biased_data'] = df
    return df

In [88]:
inject_dict = {
    'train':{
        'raw_data': snli_train,
        'bias_prevalence': 0.33,
        'bias_strength': 0.9,
        'bias_target': "neutral",
        'biased_data': None,
    },
    'dev':{
        'raw_data': snli_dev,
        'bias_prevalence': 0.95,
        'bias_strength': 0.33,
        'bias_target': "neutral",
        'biased_data': None,
    },
    'test':{
        'raw_data': snli_test,
        'bias_prevalence': 0.95,
        'bias_strength': 0.33,
        'bias_target': "neutral",
        'biased_data': None,
    }
}
biased_train = inject_data(inject_dict, 'train')
biased_dev = inject_data(inject_dict, 'dev')
biased_test = inject_data(inject_dict, 'test')

/tmp/ipykernel_2361951/3271397954.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'gold_label': 'label'}, inplace=True)
/tmp/ipykernel_2361951/3271397954.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'gold_label': 'label'}, inplace=True)
/tmp/ipykernel_2361951/3271397954.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'gold_label': 'label'}, inplace=True)


In [10]:
inject_dict_detail = {
    'train':{
        'raw_data': snli_train,
        'bias_strength': 0.5,
        'norm_strength': 0.05,
        'bias_target': "neutral",
        'biased_data': None,
    },
    'dev':{
        'raw_data': snli_dev,
        'bias_strength': 0.5,
        'norm_strength': 0.95,
        'bias_target': "neutral",
        'biased_data': None,
    },
    'test':{
        'raw_data': snli_test,
        'bias_strength': 0.5,
        'norm_strength': 0.95,
        'bias_target': "neutral",
        'biased_data': None,
    }
}
biased_train = inject_data_detail(inject_dict_detail, 'train', pathName="SNLI_detail")
biased_dev = inject_data_detail(inject_dict_detail, 'dev', pathName="SNLI_detail")
biased_test = inject_data_detail(inject_dict_detail, 'test', pathName="SNLI_detail")

/tmp/ipykernel_461908/1968711659.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'gold_label': 'label'}, inplace=True)
/tmp/ipykernel_461908/1968711659.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'gold_label': 'label'}, inplace=True)
/tmp/ipykernel_461908/1968711659.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'gold_label': 'label'}, inplace=True)


In [11]:
df = biased_train
df[['label', 'type']].value_counts(), df[['label']].value_counts(), df[['type']].value_counts()

(label          type            
 entailment     bias-independent    174214
 contradiction  bias-independent    174059
 neutral        bias-independent     91382
                bias-aligned         91382
 entailment     bias-conflicting      9202
 contradiction  bias-conflicting      9128
 Name: count, dtype: int64,
 label        
 entailment       183416
 contradiction    183187
 neutral          182764
 Name: count, dtype: int64,
 type            
 bias-independent    439655
 bias-aligned         91382
 bias-conflicting     18330
 Name: count, dtype: int64)

In [12]:
df = biased_dev
df[['label', 'type']].value_counts(), df[['label']].value_counts(), df[['type']].value_counts()

(label          type            
 entailment     bias-conflicting    3156
 contradiction  bias-conflicting    3120
 neutral        bias-independent    1618
                bias-aligned        1617
 entailment     bias-independent     173
 contradiction  bias-independent     158
 Name: count, dtype: int64,
 label        
 entailment       3329
 contradiction    3278
 neutral          3235
 Name: count, dtype: int64,
 type            
 bias-conflicting    6276
 bias-independent    1949
 bias-aligned        1617
 Name: count, dtype: int64)

In [13]:
df = biased_test
df[['label', 'type']].value_counts(), df[['label']].value_counts(), df[['type']].value_counts()

(label          type            
 entailment     bias-conflicting    3213
 contradiction  bias-conflicting    3061
 neutral        bias-independent    1610
                bias-aligned        1609
 contradiction  bias-independent     176
 entailment     bias-independent     155
 Name: count, dtype: int64,
 label        
 entailment       3368
 contradiction    3237
 neutral          3219
 Name: count, dtype: int64,
 type            
 bias-conflicting    6274
 bias-independent    1941
 bias-aligned        1609
 Name: count, dtype: int64)